In [1]:
from texascovid19 import constants
import requests
import pandas as pd
from datetime import date

Fetch each day's data.  The data is "current as of 8 p.m. the day before reporting."

In [2]:
r = requests.get(constants.DSHS_URL, verify=False)
r.raise_for_status()

/Users/dan/code/texascovid19/venv/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.dshs.state.tx.us'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [3]:
df = pd.read_html(r.text, attrs={"summary": "COVID-19 Cases in Texas Counties"})[0]

In [4]:
df.columns = ["County of Residence", "Number of Cases"]

In [5]:
df.set_index("County of Residence", inplace=True)

In [6]:
today = date.today().isoformat()

In [7]:
df.to_csv(f"{constants.DATA_PATH}/{today}.csv", header=True, index=True)

Append today's data to timeseries

In [8]:
df_timeseries = pd.read_csv(f"{constants.TIMESERIES_CSV}").set_index("County of Residence")

In [9]:
df.rename(columns={"Number of Cases": today}, inplace=True)

In [10]:
if today in df_timeseries.columns:
    df_timeseries.drop(columns=today, inplace=True)

In [11]:
df_timeseries = pd.concat([df_timeseries, df], axis=1).rename_axis('County of Residence')

In [12]:
df_timeseries = df_timeseries.fillna(0).convert_dtypes(convert_integer=True)

In [13]:
df_timeseries.to_csv(constants.TIMESERIES_CSV, header=True, index=True)